In [2]:
import pickle
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from scipy.io import savemat
import gradio as gr

# Load the trained model
with open('model2_decision_tree_ui.pkl', 'rb') as file:
    model = pickle.load(file)

# Function to make predictions and save them to a .mat file
def predict_maintenance(type_, air_temp, process_temp, rotational_speed, torque, tool_wear, id_):
    try:
        # Prepare the input features as a 2D array
        input_features = [[type_, air_temp, process_temp, rotational_speed, torque, tool_wear, id_]]

        # Predict using the model
        prediction = model.predict(input_features)

        # Load previous predictions if they exist
        try:
            data = np.load('predictions.npy')
            predictions = np.append(data, prediction)
        except FileNotFoundError:
            predictions = prediction

        # Save predictions to a .mat file
        savemat('predictions.mat', {'predictions': predictions})

        # Also save predictions to a .npy file for future appends
        np.save('predictions.npy', predictions)

        # Return the prediction
        return int(prediction[0])
    except Exception as e:
        return f"Error occurred: {str(e)}"

# Define the Gradio interface
interface = gr.Interface(
    fn=predict_maintenance,  # The prediction function
    inputs=[
        gr.Slider(0, 2, step=1, label="Type"),  # Accepts integers 0, 1, 2
        gr.Number(label="Air Temperature", precision=2),  # Accepts float
        gr.Number(label="Process Temperature", precision=2),  # Accepts float
        gr.Number(label="Rotational Speed", precision=2),  # Accepts float
        gr.Number(label="Torque", precision=2),  # Accepts float
        gr.Number(label="Tool Wear", precision=2),  # Accepts float
        gr.Number(label="ID", precision=0)  # Accepts integer
    ],
    outputs=gr.Textbox(label="Prediction"),  # The output is a text prediction
    title="Predictive Maintenance",
    description="Enter the details to predict the maintenance requirements."
)

# Launch the Gradio interface
interface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
